In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()#'vice')

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [29]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (114.96 seconds): Collecting URLs... yahoo               

n_total     26064
n_unique    25982
dtype: int64

,pubs,actual_pub
https://politico.com/magazine/story/2019/10/11/tom-steyer-2020-climate-229843,"politico, nationalreview",politico
https://politico.com/news/2019/10/13/joe-biden-rudy-giuliani-2020-045163,"politico, realclearpolitics",politico
https://politico.com/news/2019/10/15/trump-conspiracy-impeachment-defense-046284,"politico, realclearpolitics",politico
https://cnbc.com/2019/10/15/moodys-trump-on-his-way-to-an-easy-2020-win-if-economy-holds-up.html,"cnbc, realclearpolitics",cnbc
https://foxnews.com/opinion/newt-gingrich-democrats-debate-five-questions,"fox, realclearpolitics",fox
...,...,...
https://nbcnews.com/video/o-rourke-and-buttigieg-have-heated-exchange-on-guns-71343685512,"msnbc, nbcnews",nbcnews
https://nbcnews.com/politics/trump-impeachment-inquiry/pelosi-says-house-won-t-hold-vote-impeachment-time-n1066671,"msnbc, nbcnews",nbcnews
https://nbcnews.com/news/us-news/10-year-old-girl-dies-apparent-suicide-california-prompting-police-n1066516,"msnbc, nbcnews",nbcnews
https://nbcnews.com/think/opinion/kim-kardashian-s-met-gala-dress-fight-kanye-west-really-ncna1066446,"msnbc, nbcnews",nbcnews


huffpost          80
cnn              835
investing.com    886
politico         212
time              16
                ... 
arirang           56
ccn                0
cointelegraph     73
cryptocoin        43
cryptonews       325
Length: 73, dtype: int64

In [30]:
collect_results[2]['thinkprogress']

82

# 2. 모은 URL중 이미 받은 것은 버린다

In [31]:
select_results = crawler.select(); display(*select_results)

100.00% (2.97 seconds): Selecting URLs... cryptonews          

n_total     723
n_unique    723
dtype: int64

,pubs,actual_pub


huffpost           1
cnn                8
investing.com    104
politico           1
time               0
                ... 
arirang            1
ccn                0
cointelegraph      0
cryptocoin         0
cryptonews         1
Length: 73, dtype: int64

In [32]:
crawler.urls_selected['thinkprogress']

set()

# 3. 해당 URL의 뉴스를 다운로드한다

In [33]:
download_results = crawler.download(); download_results

8.44% (13.28 seconds): Downloading... blastingnews        

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


32.64% (47.34 seconds): Downloading... indiatimes          

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100.00% (369.58 seconds): Downloading... yahoo               

,downloaded,trashed
huffpost,0,1
cnn,1,7
investing.com,45,59
politico,1,0
time,0,0
...,...,...
arirang,1,0
ccn,0,0
cointelegraph,0,0
cryptocoin,0,0


In [14]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

[]